# Petroleum Blending (Solved)
## Problem Definition
Harry Stamper is an engineer working on a petroleum company. The company produces three grades of motor oil – super, premium, and extra. The three grades are made from the same 3 ingredients, named component 1, component 2 and component 3. The company wants to determine the optimal mix of the 3 components in each grade of motor oil that will maximize profit. 

The maximum availability of each component and their cost per barrel are as follows:

| Component    | Max barrels available / day  | Cost / barrel |
|--------------|------------------------------|---------------|
| Component 1  | 4.500                        | 12€           |
| Component 2  | 2.700                        | 10€           |
| Component 3  | 3.500                        | 14€           |

To ensure the appropriate blend, each grade must have a minimum amount of component 1 plus a combination of other components as follows:

| Grade   | Component 1  | Component 2      | Component 3       | Selling price/barrel |
|---------|--------------|------------------|-------------------|-----------------------|
| Super   | At least 50% | No more than 30% | -                 | 23€                   |
| Premium | At least 40% | -                | No more than 25% | 20€                   |
| Extra   | At least 60% | At least 10%     | -                 | 18€                   |

The company wants to make at least 3000 barrels of each blend.
*Formulate a LP to help Harry Stamper find the optimal blend that maximises profit.*

## Problem Model
The decision variables are: 

$x_{ij}$ = amount in barrels of component i in grade j

Objective function

max $z = 23*(x_{11} + x_{21} + x_{31}) + 20*(x_{12} + x_{22} + x_{32})+18*(x_{13} + x_{23} + x_{33})-12*(x_{11} + x_{12} + x_{13}) - 10*(x_{21} + x_{22} + x_{23})-14*(x_{31} + x_{32} + x_{33})$

Constraints

$x_{11} \geq 0.5*(x_{11} + x_{21} + x_{31})$ Component 1 requirement in Super

$x_{21} \leq 0.3*(x_{11} + x_{21} + x_{31})$ Component 2 requirement in Super

$x_{12} \geq 0.4*(x_{12} + x_{22} + x_{32})$  Component 1 requirement in Premium

$x_{32} \leq 0.25*(x_{12} + x_{22} + x_{32})$ Component 3 requirement in Premium

$x_{13} \geq 0.6*(x_{13} + x_{23} + x_{33})$ Component 1 requirement in Extra

$x_{23} \geq 0.1*(x_{13} + x_{23} + x_{33})$ Component 2 requirement in Extra


$x_{11}+x_{12}+x_{13} \leq 4500$  Component 1 availability

$x_{21}+x_{22}+x_{23} \leq 2700$  Component 1 availability

$x_{31}+x_{32}+x_{33} \leq 3500$ Component 1 availability

$x_{11}+x_{21}+x_{31} \geq 3000$  Super minimum production

$x_{12}+x_{22}+x_{32} \geq 3000$  Premium minimum production

$x_{13}+x_{23}+x_{33} \geq 3000$ Extra minimum production

**Analysis questions**

* Which is the total profit obtained by the company with the Extra grade with the solution provided by the solver?

Just multiply the value of every basic (non-zero) decision variable times the objective function coefficient. 

* Which is the maximum value that the profit per barrel of component 2 in the Extra grade can take without changing the base of the optimal solution?

The profit per barrel of component two in the extra grade is 8.00 (look at the second last row of the first table). The maximum value that it can take is the upper bound of the objective function in that row, which is 25. The profit per barrel of component 2 could change from 8.00 to 25 without changing the base. Since there are no changes in the constraint, there are no changes in the feasible region and the optimal solution will be the same point and same quantities. Therefore if we were able to raise the profit by 1 eur per barrel of component 2 in extra grade, the total profit (objective function) will increase 12000 (because this is the quantity of this component in the extra grade in the optimal solution).

* If the cost of the barrel of component 3 increased 1€, will the optimal composition change? Motivate your response

In this case an increase of the cost of barrel of component 3 means that the profit per component 3 in the three grades is going to decrease by 1 eur. Since the lower bound of the profit per component 3 in the “Super” grade is equal to the objective function coefficient, there will be a change in the optimal composition. 


* Do we use every barrel available of every component in the optimal solution? Or do we have any surplus availability for any component? Motivate your response

If we look at the components availability, we can clearly see that we have a slack of 1700 barrels of component 3.


* The provider of Component 2 may have a problem with the delivery and the availability may be reduced in 400 barrels. Will this shortage affect the optimal composition? Motivate your response*

If the component 2 is short in 4000 barrels (from 2700 to 2300) the availability will still be higher than the lower bound of the component 2 available constraint which is 2250, so this shortage in principle will not affect the optimal solution.

* Grace wants to use the results to renegotiate the terms and conditions with the suppliers of the different components. Which component is a better candidate to increase the availability? How would you use the model to negotiate new terms with the provider, assuming you do not want any change in the base of the optimal solution? Motivate your response

Now, this is a quite an interesting question. If we were to increase the availability, we can first discard component 3, because we have the aforementioned slack, which means that we are not using all the available material, so the candidates are component 1 and 2. Now, comparing both components, component 1 has a much larger shadow price, which means that for every change of one unit in the right hand side, we will have an increase of 20 eur in the profit. So this is definitively the best candidate. We could try to negotiate the terms and conditions and try to tell the provider that we want more units of this component. We could use up to 6200 units (upper right hand side) without changes in the optimal composition. Now, since we are buying more units, we may as well ask for a lower price. If we look at the profits of component 1 in the first table, we could increase the profits of Premium, component 1 by 3 euros (from 8 to 11 which is the upper bound) so we could try to renegotiate the price from 12 down to 9. 